In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('recommender').getOrCreate()

24/11/03 02:38:34 WARN Utils: Your hostname, AlienEE resolves to a loopback address: 127.0.1.1; using 10.3.0.66 instead (on interface wlp4s0)
24/11/03 02:38:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 02:38:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
dataset =  spark.read.csv('datasets/movielens_ratings.csv', inferSchema=True, header=True)

In [5]:
dataset.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [6]:
from pyspark.ml.recommendation import ALS

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator

In [9]:
dataset.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [11]:
dataset.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [12]:
train_data, test_data = dataset.randomSplit([0.8, 0.2])

In [15]:
als = ALS(maxIter=5, regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [16]:
model = als.fit(train_data)

In [17]:
predictions = model.transform(test_data)

In [19]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      1|   1.0|    28|   2.6600735|
|      0|   1.0|    26|   2.7078896|
|      1|   1.0|    26|   3.0936084|
|      2|   1.0|    26|   3.1366713|
|      5|   2.0|    26|   3.0139832|
|      4|   1.0|    12|   1.0220032|
|      3|   2.0|    22|  0.86305577|
|      4|   2.0|     1|   3.0408328|
|      1|   1.0|     3|-0.019099437|
|      0|   1.0|    19|    0.675124|
|      1|   4.0|    15|    1.531164|
|      2|   1.0|    15|     2.62296|
|      0|   1.0|     8|   1.6565083|
|      1|   1.0|     7|  0.88941073|
|      4|   1.0|    24| -0.60542387|
|      2|   4.0|    21|   2.3139622|
|      0|   1.0|    11|   1.8380736|
|      4|   1.0|    14|    2.096154|
|      5|   2.0|     0|   1.2727236|
|      1|   1.0|    18|  0.52977294|
+-------+------+------+------------+
only showing top 20 rows



In [20]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [21]:
rmse = evaluator.evaluate(predictions)

In [22]:
print(f"RMSE = {rmse}")

RMSE = 1.5913608768864713


In [24]:
single_user = test_data.filter(test_data['userId'] == 11).select(['movieId', 'userId'])

In [25]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     10|    11|
|     12|    11|
|     16|    11|
|     40|    11|
|     45|    11|
|     48|    11|
|     50|    11|
|     72|    11|
|     75|    11|
+-------+------+



In [26]:
recommendations = model.transform(single_user)

In [27]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     50|    11|   3.883024|
|     10|    11|  3.6803646|
|     16|    11|  3.3324566|
|     48|    11|  3.3289158|
|     12|    11|  1.9158751|
|      0|    11|  1.8380736|
|     75|    11|  1.6256001|
|     45|    11|  0.5376617|
|     72|    11|-0.27613038|
|     40|    11|-0.57283735|
+-------+------+-----------+

